<a href="https://colab.research.google.com/github/soaminfosec/MSDefnder_Pull_request/blob/main/Inventorycompare2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Install required packages
!pip install -q pandas openpyxl

import pandas as pd
from google.colab import files
from io import BytesIO
import io

# Function to load CSV or Excel
def load_file(file_dict):
    if not file_dict:
        return None, None
    name = list(file_dict.keys())[0] # Get the first (and only) uploaded file name
    ext = name.split('.')[-1].lower()
    if ext == 'csv':
        return pd.read_csv(io.BytesIO(file_dict[name])), name
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(io.BytesIO(file_dict[name])), name
    return None, None

# Upload files one by one
print("📤 Please upload the Inventory file (Required) with columns: 'COMPUTER NAME' or 'Computer Name' or 'Endpoint' or 'Name', 'IP', 'Status':")
uploaded_inventory = files.upload()
inventory_df, _ = load_file(uploaded_inventory)

print("\n📤 Please upload the First AV Product file (Required) with columns: 'COMPUTER NAME' or 'Computer Name' or 'Endpoint' or 'Name', and any column with 'IP' in its name:")
uploaded_av1 = files.upload()
av1_df, _ = load_file(uploaded_av1)

print("\n📤 Please upload the Second AV Product file (Optional, press 'cancel' if not available) with columns: 'COMPUTER NAME' or 'Computer Name' or 'Endpoint' or 'Name', and any column with 'IP' in its name:")
uploaded_av2 = files.upload()
av2_df, _ = load_file(uploaded_av2) if uploaded_av2 else (None, None)


# Normalize computer name from known columns
def extract_computer_name(row):
    for col in ['COMPUTER NAME', 'Computer Name', 'Endpoint', 'Name']:
        if col in row and pd.notna(row[col]):
            return str(row[col]).strip().lower()
    return ""

# Normalize IP from known columns
def extract_ip(row):
    for col in row.index:
        if 'ip' in col.lower() and pd.notna(row[col]):
            return str(row[col]).strip()
    return ""

# Match computer name in any known columns
def find_match_by_name(df, name_to_match):
    for col in ['COMPUTER NAME', 'Computer Name', 'Endpoint', 'Name']:
        if col in df.columns:
            matched = df[df[col].astype(str).str.strip().str.lower() == name_to_match]
            if not matched.empty:
                return matched.iloc[0], f"Matched by {col}"
    return None, "No match by name"

# Match inventory IP to any IP column in AV product (supports multiple IPs in AV1)
def find_match_by_ip(av_df, ip_to_match, label='AV'):
    if av_df is None:
        return None, f"No match by IP in {label} (data not available)"
    ip_columns = [col for col in av_df.columns if 'ip' in col.lower()]
    for col in ip_columns:
        matched = av_df[av_df[col].astype(str).str.strip() == ip_to_match]
        if not matched.empty:
            return matched.iloc[0], f"Matched by IP ({col}) in {label}"
    return None, f"No match by IP in {label}"

# Main comparison function
def compare(inventory_df, av1_df, av2_df=None):
    results = []

    for _, row in inventory_df.iterrows():
        inv_name = extract_computer_name(row)
        inv_ip = extract_ip(row)
        inv_status = row.get('Status', '')

        # First AV match
        av1_row, av1_match_type = find_match_by_name(av1_df, inv_name)
        if av1_row is None:
            av1_row, av1_match_type = find_match_by_ip(av1_df, inv_ip, "First AV")

        # Second AV match (optional)
        if av2_df is not None:
            av2_row, av2_match_type = find_match_by_name(av2_df, inv_name)
            if av2_row is None:
                av2_row, av2_match_type = find_match_by_ip(av2_df, inv_ip, "Second AV")

            av2_name = extract_computer_name(av2_row) if av2_row is not None else ""
        else:
            av2_match_type = "Second AV not uploaded"
            av2_name = ""


        # Determine reporting status
        if av1_row is not None and av2_df is not None and av2_row is not None:
            overall_status = "✅ Reporting to both AVs"
        elif av1_row is not None:
            overall_status = "⚠️ Reporting only to First AV"
        elif av2_df is not None and av2_row is not None:
            overall_status = "⚠️ Reporting only to Second AV"
        else:
            overall_status = "❌ Not reporting to any AV"

        result_row = {
            'Inventory Name': inv_name.upper(),
            'Inventory IP': inv_ip,
            'Status': inv_status,
            'First AV Match': av1_match_type,
            'Overall Reporting Status': overall_status
        }

        # Add all columns from av1_row if a match was found
        if av1_row is not None:
            for col in av1_df.columns:
                 result_row[f'First AV - {col}'] = av1_row.get(col, '')


        if av2_df is not None:
            result_row['Second AV Name'] = av2_name.upper()
            result_row['Second AV Match'] = av2_match_type

        results.append(result_row)

    return pd.DataFrame(results)

# Run comparison
final_df = compare(inventory_df, av1_df, av2_df)

# Export to Excel
excel_filename = 'AV_Comparison_Result.xlsx'
excel_bytes = BytesIO()
with pd.ExcelWriter(excel_bytes, engine='openpyxl') as writer:
    final_df.to_excel(writer, index=False, sheet_name='AV_Comparison_Result')
excel_bytes.seek(0)

# Save the BytesIO object to a file before downloading
with open(excel_filename, 'wb') as f:
    f.write(excel_bytes.getvalue())

# Download result
files.download(excel_filename)

📤 Please upload the Inventory file (Required) with columns: 'COMPUTER NAME' or 'Computer Name' or 'Endpoint' or 'Name', 'IP', 'Status':


Saving IOCL_Inventory.xlsx to IOCL_Inventory (10).xlsx

📤 Please upload the First AV Product file (Required) with columns: 'COMPUTER NAME' or 'Computer Name' or 'Endpoint' or 'Name', and any column with 'IP' in its name:


Saving computer_status_export_D8246D0CBEE742D98B7953D8AB7BE256 DR 133 26 July.csv to computer_status_export_D8246D0CBEE742D98B7953D8AB7BE256 DR 133 26 July.csv

📤 Please upload the Second AV Product file (Optional, press 'cancel' if not available) with columns: 'COMPUTER NAME' or 'Computer Name' or 'Endpoint' or 'Name', and any column with 'IP' in its name:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>